<a href="https://colab.research.google.com/github/ehsankarami1358/LOKA_HYDRO/blob/main/Digital_Hillchart_LOKA_U2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor

In [20]:
# ============================================
# 1. LOAD AND PREPARE DATA
# ============================================

print("="*60)
print("HILL CHART ANALYSIS - UNIT 2")
print("="*60)

# Load your CSV file
df = pd.read_csv("u2_MW_OP_FL_L_1_1_2026_11_2_2026_R3.csv")

# Convert timestamp
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Calculate Net Head
df['Net_Head'] = df['HEAD_L(m)'] - df['TAIL_L(m)']

# Rename columns for easier use
df.rename(columns={
    'ACTIVE_POWER(MW)': 'Power',
    'OPPENING(%)': 'Opening',
    'FLOW(m3/s)': 'Flow',
    'SPEED(RPM)': 'Speed',
    }, inplace=True)

# Calculate efficiency
df['Efficiency'] = (df['Power'] * 1000) / (9.81 * df['Flow'] * df['Net_Head'])



HILL CHART ANALYSIS - UNIT 2


In [21]:
RHO = 1000.0          # kg/m3 (use 999.3 if you want)
G = 9.81              # m/s2

def add_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Feature engineering for a digital hill chart model.
    Keep it simple and physically meaningful.
    """
    df = df.copy()

    # Basic nonlinearities / interactions, using the renamed columns
    df["QH"] = df["Flow"] * df["Net_Head"]
    df["Q_sqrtH"] = df["Flow"] * np.sqrt(np.clip(df["Net_Head"], 1e-6, None))
    df["Q2"] = df["Flow"] ** 2
    df["H2"] = df["Net_Head"] ** 2
    df["Open2"] = df["Opening"] ** 2
    df["Speed2"] = df["Speed"] ** 2

    return df

In [22]:
def compute_efficiency_from_power(df: pd.DataFrame, power_col: str) -> pd.Series:
    """
    Compute total efficiency using Power, Flow, Head.
    Power in MW, Q in m3/s, H in m.
    """
    P_watts = df[power_col].astype(float) * 1e6
    denom = (RHO * G * df["Flow"].astype(float) * df["Net_Head"].astype(float)).replace(0, np.nan)
    return (P_watts / denom)

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Basic sanity filters. Adjust thresholds to your plant.
    """
    df = df.copy()

    # Drop missing required fields, using the renamed columns
    req = ["Net_Head", "Flow", "Opening", "Power"]
    df = df.dropna(subset=req)
    df["Eta_meas"] = compute_efficiency_from_power(df, "Power")
    return df

def train_digital_hillchart_power_model(df: pd.DataFrame):
    df = clean_data(df)
    df = add_features(df)

    feature_cols = [
     'Power',
    'Opening',
    'Flow',
    'Speed',
    'Net_Head'
    ]

    X = df[feature_cols]
    y = df["Power"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = XGBRegressor(
        n_estimators=700,
        learning_rate=0.03,
        max_depth=5,
        subsample=0.85,
        colsample_bytree=0.85,
        reg_lambda=1.0,
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train, y_train)

    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)

    print(f"Test MAE (MW): {mae:.3f}")
    print(f"Test R2:       {r2:.4f}")

    return model, feature_cols, df

In [23]:
def score_and_export(df_processed: pd.DataFrame, model, feature_cols, out_xlsx="digital_hillchart_results.xlsx"):
    df = df_processed.copy()

    df = clean_data(df)
    df = add_features(df)

    df["Power_pred_MW"] = model.predict(df[feature_cols])

    # Efficiency measured vs predicted - use 'Efficiency' column
    df["Eta_pred"] = compute_efficiency_from_power(df, "Power_pred_MW")
    df["DegradationIndex"] = df["Efficiency"] - df["Eta_pred"]   # negative = worse than expected

    # Optional: normalize degradation (percent of expected)
    df["DegradationPct"] = 100 * df["DegradationIndex"] / df["Eta_pred"].replace(0, np.nan)

    # Summary by month (using 'Timestamp' column)
    writer = pd.ExcelWriter(out_xlsx, engine="openpyxl")
    df.to_excel(writer, sheet_name="scored_points", index=False)

    if "Timestamp" in df.columns:
        dt = pd.to_datetime(df["Timestamp"], errors="coerce")
        df["Month"] = dt.dt.to_period("M").astype(str)
        month_summary = df.groupby("Month").agg(
            n=("Power", "size"),
            P_mean=("Power", "mean"),
            H_mean=("Net_Head", "mean"),
            Q_mean=("Flow", "mean"),
            open_mean=("Opening", "mean"),
            eta_meas_mean=("Efficiency", "mean"),
            eta_pred_mean=("Eta_pred", "mean"),
            degr_mean=("DegradationIndex", "mean"),
            degr_p05=("DegradationIndex", lambda s: np.quantile(s, 0.05)),
            degr_p95=("DegradationIndex", lambda s: np.quantile(s, 0.95))
        ).reset_index()
        month_summary.to_excel(writer, sheet_name="monthly_summary", index=False)

    writer.close()
    print(f"Saved: {out_xlsx}")
    return df

if __name__ == "__main__":
    # Load the raw data
    df_processed = pd.read_csv("u2_MW_OP_FL_L_1_1_2026_11_2_2026_R3.csv")

    # Apply initial processing steps to df_processed
    df_processed['Timestamp'] = pd.to_datetime(df_processed['Timestamp'])
    df_processed['Net_Head'] = df_processed['HEAD_L(m)'] - df_processed['TAIL_L(m)']
    df_processed.rename(columns={
        'ACTIVE_POWER(MW)': 'Power',
        'OPPENING(%)': 'Opening',
        'FLOW(m3/s)': 'Flow',
        'SPEED(RPM)': 'Speed',
        }, inplace=True)
    df_processed['Efficiency'] = (df_processed['Power'] * 1000) / (9.81 * df_processed['Flow'] * df_processed['Net_Head'])

    # Train the model
    model, feature_cols, df_train = train_digital_hillchart_power_model(df_processed)
    # Score and export results
    scored = score_and_export(df_processed, model, feature_cols, out_xlsx="digital_hillchart_results.xlsx")

Test MAE (MW): 0.749
Test R2:       0.9940
Saved: digital_hillchart_results.xlsx
